In [16]:
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
import math
import numpy as np
from itertools import combinations

In [17]:
df = pd.read_csv("data/imports-85-clean.csv")
df = df.loc[df["price"] != '?']
df = df.astype({"price": float}, errors='raise')
sample_list = df.drop(columns=["price"]).columns

In [18]:
msk = np.random.rand(len(df)) < 0.9
train = df[msk]
test_cv = df[~msk]
msk_cv = np.random.rand(len(test_cv)) < 0.7
cv = test_cv[msk_cv]
test = test_cv[~msk_cv]

In [19]:
list_combinations = list(combinations(sample_list, len(sample_list) - 1))
best_best_score = 0
best_best_k = -1
best_best_c = None
while len(list_combinations) > 0:

    max_score = 0
    best_k = -1
    best_c = None
    scores = []

    for c in list_combinations:
        if not c:
            continue
        for i in range(1, 6):
            neigh = KNeighborsRegressor(n_neighbors=i)
            x_train = train[np.asarray(c)]
            y_train = train["price"]
            x_cv = cv[np.asarray(c)]
            y_cv = cv["price"]
            neigh.fit(x_train, y_train)

            y_pred = neigh.predict(x_cv)
            score = neigh.score(x_cv, y_cv)
            scores += (c, i, score)
            if score > max_score:
                max_score = score
                best_k = i
                best_c = c


    if best_c is None:
        break

    print(max_score)
    print(best_k)
    print(best_c)
    print(len(best_c))

    if max_score > best_best_score:
        best_best_score = max_score
        best_best_k = best_k
        best_best_c = best_c
    list_combinations = list(combinations(best_c, len(best_c) - 1))

0.7261004828192383
1
('symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg')
24
0.7644182878743029
1
('symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'num-of-cylinders', 'engine-size', 'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg')
23
0.8163583554297916
2
('symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'num-of-cylinders', 'engine-size', 'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm

In [20]:
print(best_best_c)
print(best_best_k)
print(best_best_score)

('make', 'aspiration', 'drive-wheels', 'width', 'curb-weight', 'city-mpg')
2
0.8965304319551168


In [21]:
neigh = KNeighborsRegressor(n_neighbors=best_best_k)
x_train = train[np.asarray(best_best_c)]
y_train = train["price"]
x_test = test[np.asarray(best_best_c)]
y_test = test["price"]
neigh.fit(x_train, y_train)
y_pred = neigh.predict(x_test)
print(neigh.score(x_test, y_test))

0.9464018999299919
